# LCS Benchmark Injection Testing

## Setup

In [19]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
%run "../../Setup_Scripts/Setup_Generic.ipynb"
scope.adc.samples = 900

INFO: Found ChipWhisperer😍


In [ ]:
scope.dis()
target.dis()

In [2]:
#import necessary library 
import linear_comp as lc
import matplotlib.pyplot as plt 
import chipwhisperer as cw
import numpy as np

In [3]:
def cap_pass_trace():
    scope.arm()
    ret = scope.capture()
    num_char = target.in_waiting()
    while num_char > 0:
        target.read(num_char, 10)
        time.sleep(0.01)
        num_char = target.in_waiting()

    trace = scope.get_last_trace()
    return trace


<h3> This is for Research Benchmark Checking </h3>

In [24]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../hardware/victims/firmware/LCS
gmake PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2 -j

SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
gmake[1]: '.dep' is up to date.
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
.
Welcome to another exciting ChipWhisperer target build!!
avr-gcc (Homebrew AVR GCC 9.4.0) 9.4.0
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling:
-en     .././hal/xmega/XMEGA_AES_driver.c ...
-e Done!
.
Compiling:
-en     .././hal/xmega/xmega_hal.c ...
-e Done!
.
Compiling:
-en     .././hal/xmega/uart.c ...
-e Done!
.
Compiling:
-en     .././hal/xmega/usart_driver.c ...
-e Done!
.
Compiling:
-en     LCS.c ...
-e Done!
.
LINKING:
-en     LCS-CWLITEXMEGA.elf ...
-e Done!
.
Creating load file for Flash: LCS-CWLITEXMEGA.bin
avr-objcopy -O binary -R .eeprom -R .fuse -R .lock -R .signature LCS-CWLITEXMEGA.elf LCS-CWLITEXMEGA.bin
.
Creating load file

In [25]:
cw.program_target(scope, prog, "../../hardware/victims/firmware/LCS/LCS-{}.hex".format(PLATFORM))

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 2105 bytes


In [ ]:
#reset_target(scope)
trace_comp = cap_pass_trace()
#cw.plot(trace_comp)

In [7]:
trace_set=[]
reset_target(scope)
for i in range(240):
    trace_hit = cap_pass_trace()
    if(len(trace_set))==0:
        trace_set.append(trace_hit)
    else:
        flag=False
        for j in range(len(trace_set)):
            if(lc.linear_comp(trace_hit,trace_set[j],0.1,150))>850:
                flag=True
        
        if not flag:
            trace_set.append(trace_hit)
    
print(len(trace_set))

4


In [ ]:
cw.plot(trace_set[0]) * cw.plot(trace_set[4])
#cw.plot(trace_comp) * cw.plot(trace_set[0])
#print(lc.linear_comp(trace_comp,trace_set[0],0.1,550))


#cw.plot(trace_set[0])* cw.plot(trace_set[1]) 
#abs(np.sum(trace_set[0]-trace_set[1]))

#cw.plot(trace_set[0])*cw.plot(trace_set[1])*cw.plot(trace_set[2])*cw.plot(trace_set[3])*cw.plot(trace_set[4])

# Testing Part for LCS

In [26]:
result=[]

for i in range(4): # we now are getting right result
    reset_target(scope)
    for j in range(256):
        trace_test=cap_pass_trace()
        x=max([lc.linear_comp(trace_test,trace_set[k],0.1,150) for k in range(len(trace_set))])
        result.append(x)

In [27]:
with open("LCS_Accuracy.csv",'w') as f:
    f.write("Test Iteration,Matching,Decision\n")
    for i in range(len(result)):
        if(result[i]>600):
            f.write(f"{i},{result[i]},OK\n")
        else:
            f.write(f"{i},{result[i]},Failed\n")